In [ ]:
# =========================================
# Load and Inspect Fuzzy Patient CSV Data
# =========================================

import numpy as np
import pandas as pd

# --- Configuration ---
# Input CSV path (fix double .csv extension)
input_csv_path = "csv_and_figures/AGBRESA_fuzzy_patient_data.csv"

# --- Load Data ---
df_ratio = pd.read_csv(input_csv_path)

# --- Inspect Data ---
print("✅ Loaded fuzzy patient dataset")
print(f"Shape: {df_ratio.shape}")
print("\nPreview:")
print(df_ratio.head())

# If you want to display directly (in Jupyter)
df_ratio


In [ ]:
import numpy as np
import pandas as pd

# --- Transform fat ratio data ---
def transform_df_ratio(df):
    class_map = {1: "Right_LES", 2: "Left_LES"}
    df["volume_class"] = df["class"].map(class_map)
    dfs = []
    for col, val in [("auto_fat_ratio", 1), ("manual_fat_ratio", 0)]:
        tmp = df[["pid", "volume_class", col]].copy()
        tmp.rename(columns={col: "ratio"}, inplace=True)
        tmp["ai_derived"] = val
        tmp["MRI_id"] = tmp["pid"].str.extract(r"(ID.*)")
        tmp["judge"] = np.where(val == 1, tmp["volume_class"] + "_ai", tmp["volume_class"] + "_gd")
        tmp["exam"] = tmp["MRI_id"]
        tmp["rating"] = tmp["ratio"]
        tmp["patient_id"] = tmp["pid"].str.extract(r"([A-Za-z0-9]+)")
        dfs.append(tmp)
    return pd.concat(dfs, ignore_index=True)

# --- Transform volume data ---
def transform_df_volume(df):
    class_map = {1: "Right_LES", 2: "Left_LES"}
    df["volume_class"] = df["class"].map(class_map)
    dfs = []
    for col, val in [("auto_volume", 1), ("manual_volume", 0)]:
        tmp = df[["pid", "volume_class", col]].copy()
        tmp.rename(columns={col: "ratio"}, inplace=True)
        tmp["ai_derived"] = val
        tmp["MRI_id"] = tmp["pid"].str.extract(r"(ID.*)")
        tmp["judge"] = np.where(val == 1, tmp["volume_class"] + "_ai", tmp["volume_class"] + "_gd")
        tmp["exam"] = tmp["MRI_id"]
        tmp["rating"] = tmp["ratio"]
        tmp["patient_id"] = tmp["pid"].str.extract(r"([A-Za-z0-9]+)")
        dfs.append(tmp)
    return pd.concat(dfs, ignore_index=True)

# Example
df_transformed_ratio = transform_df_ratio(df_ratio)
df_transformed_volume = transform_df_volume(df_ratio)

print(df_transformed_ratio.head())
print(df_transformed_volume.head())


In [ ]:
import os
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from scipy.stats import t, chi2
import warnings

warnings.filterwarnings("ignore")

# ==========================
# Parameters
# ==========================
alpha = 0.05
equiv_pct = 0.10        # Equivalence threshold (±10%)
MIN_SUBJECTS = 3
MIN_OBSERVATIONS = 10

# Use transformed data
df = df_transformed_volume.copy()
# df = df_transformed_ratio.copy()

# ==========================
# Helper Functions
# ==========================
def extract_time_point(mri_id: str) -> str:
    """Extract visit time point from MRI ID."""
    if "_R13" in mri_id: return "R13"
    if "_R0" in mri_id:  return "R0"
    if "_BDC" in mri_id: return "BDC"
    return "Unknown"

def min_equiv_margin(beta, se, df, ref_mean, alpha=0.05):
    """Compute minimal equivalence margin (%) from CI of beta."""
    half_ci = t.ppf(1 - alpha, df) * se
    ci_low, ci_high = beta - half_ci, beta + half_ci
    margin_abs = max(abs(ci_low), abs(ci_high))
    return margin_abs, margin_abs / abs(ref_mean) * 100

def run_lrt(model1, model2):
    """Likelihood ratio test between two models."""
    lr_stat = 2 * (model2.llf - model1.llf)
    df_diff = model2.df_modelwc - model1.df_modelwc
    p_val = chi2.sf(lr_stat, df_diff)
    return lr_stat, df_diff, p_val

# Add time point column
df["time_point"] = df["MRI_id"].apply(extract_time_point)

# ==========================
# Main Analysis Loop
# ==========================
results = []

for cls, sub in df.groupby("volume_class"):
    n_subj = sub["patient_id"].nunique()
    n_obs = len(sub)

    # Skip if too few subjects or obs
    if n_subj < MIN_SUBJECTS or n_obs < MIN_OBSERVATIONS or sub["ai_derived"].nunique() < 2:
        continue

    # Candidate models
    formulas = [
        "ratio ~ C(ai_derived)",
        "ratio ~ C(ai_derived) + C(time_point)"
    ]
    models = []

    # Fit models
    for f in formulas:
        try:
            m = smf.mixedlm(f, sub, groups="patient_id", re_formula="1")
            res = m.fit(reml=False, maxiter=1000)
            models.append((f, res))
            print(f"{cls}: ✅ Model fitted: {f}")
        except Exception as e:
            print(f"{cls}: ❌ Model failed: {f}, Reason: {e}")

    if not models:
        continue

    # Model selection by LRT
    f0, r0 = models[0]
    for f1, r1 in models[1:]:
        try:
            stat, df_diff, p = run_lrt(r0, r1)
            print(f"{cls}: LRT {f0} vs {f1}, χ²={stat:.2f}, p={p:.4f}")
            if p < 0.05:  # Prefer richer model
                f0, r0 = f1, r1
        except Exception as e:
            print(f"{cls}: LRT failed, Reason: {e}")

    formula_used, res = f0, r0

    # Skip if no AI effect term
    if "C(ai_derived)[T.1]" not in res.params:
        continue

    # --- ICC ---
    var_u = res.cov_re.iloc[0, 0]
    var_e = res.scale
    icc = var_u / (var_u + var_e)
    avg_per_sub = n_obs / n_subj
    se_icc = np.sqrt((2 * icc**2 * (1 - icc)**2) / (n_subj * (avg_per_sub - 1)))
    icc_ci = t.ppf([0.025, 0.975], res.df_resid) * se_icc + icc
    icc_l, icc_h = max(0, icc_ci[0]), min(1, icc_ci[1])

    # --- Effect size (beta) ---
    beta = res.params["C(ai_derived)[T.1]"]
    se = res.bse["C(ai_derived)[T.1]"]
    df_d = res.df_resid
    ci_half = t.ppf(1 - alpha, df_d) * se
    ci_low, ci_high = beta - ci_half, beta + ci_half

    # --- Group means ---
    m_man, s_man = sub.loc[sub.ai_derived == 0, "ratio"].agg(["mean", "std"])
    m_ai, s_ai = sub.loc[sub.ai_derived == 1, "ratio"].agg(["mean", "std"])

    # --- TOST ±10% ---
    delta10 = equiv_pct * abs(m_man)
    t1, t2 = (beta + delta10) / se, (beta - delta10) / se
    p_10 = max(t.sf(t1, df_d), t.cdf(t2, df_d))

    # --- TOST minimal margin ---
    min_abs, min_pct = min_equiv_margin(beta, se, df_d, m_man, alpha)
    t1_min, t2_min = (beta + min_abs) / se, (beta - min_abs) / se
    p_min = max(t.sf(t1_min, df_d), t.cdf(t2_min, df_d))

    # --- Save results ---
    results.append({
        "Muscle Region": cls,
        "Manual (Mean±SD)": f"{m_man:.4f} ± {s_man:.4f}",
        "AI (Mean±SD)": f"{m_ai:.4f} ± {s_ai:.4f}",
        "Diff [90% CI]": f"{beta:.4f} [{ci_low:.4f}, {ci_high:.4f}]",
        "Min Equiv Margin (%)": round(min_pct, 2),
        "TOST p (±10%)": round(p_10, 6),
        "TOST p (min margin)": round(p_min, 10),
        "ICC": round(icc, 3),
        "ICC 95% CI": f"[{icc_l:.2f}, {icc_h:.2f}]",
        "Model": f"{formula_used} + (1 | patient_id)"
    })

# ==========================
# Summary Table
# ==========================
results_df = pd.DataFrame(results)[[
    "Muscle Region", "Manual (Mean±SD)", "AI (Mean±SD)",
    "Diff [90% CI]", "Min Equiv Margin (%)",
    "TOST p (±10%)", "TOST p (min margin)",
    "ICC", "ICC 95% CI", "Model"
]]

print("\n✅ Final Results")
results_df.head()


In [ ]:
import pandas as pd
import numpy as np

# ================================
# Mean Absolute Error (MAE) by Muscle Class
# ================================

mae_results = []

for cls, sub in df.groupby("volume_class"):
    # Pivot: patient_id × [manual, AI]
    merged = sub.pivot_table(
        index="patient_id",
        columns="ai_derived",
        values="ratio"
    )
    merged.columns = ["manual", "ai"]  # 0 → manual, 1 → AI

    # Compute MAE
    mae = np.mean(np.abs(merged["manual"] - merged["ai"]))
    mae_results.append({"volume_class": cls, "MAE": mae})

# Convert to DataFrame
mae_df = pd.DataFrame(mae_results)

print("\n✅ MAE Results by Volume Class")
print(mae_df)


In [ ]:
# =====================================
# Bland–Altman Analysis (Classic vs Model-based)
# =====================================

import os
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

# -----------------------------
# Config
# -----------------------------
plot_dir = 'nnsam2_results/csv_and_plots/Dixon'
os.makedirs(plot_dir, exist_ok=True)

# Choose transformed DataFrame
# df = df_transformed_volume.copy()
df = df_transformed_ratio.copy()

# Extract time points
df['time_point'] = df['MRI_id'].str.extract(r'_(BDC|R0|R13)')[0]
regions = df['volume_class'].unique()

ba_results = []

# -----------------------------
# Global plotting ranges (to keep figures comparable)
# -----------------------------
all_means, all_diffs = [], []
for region in regions:
    sub = df[df['volume_class'] == region]
    pivot = sub.pivot_table(index=['patient_id', 'time_point'],
                            columns='ai_derived', values='ratio').dropna()
    if pivot.shape[0] < 3:
        continue
    pivot.columns = ['Manual', 'AI']
    pivot['diff'] = pivot['AI'] - pivot['Manual']
    pivot['mean'] = pivot[['Manual', 'AI']].mean(axis=1)
    all_means.extend(pivot['mean'])
    all_diffs.extend(pivot['diff'])

x_margin = (max(all_means) - min(all_means)) * 0.1
y_margin = (max(all_diffs) - min(all_diffs)) * 0.2
global_xlim = (min(all_means) - x_margin, max(all_means) + x_margin)
global_ylim = (min(all_diffs) - y_margin, max(all_diffs) + y_margin)

# -----------------------------
# Per-region analysis
# -----------------------------
for region in regions:
    sub = df[df['volume_class'] == region]

    # Pivot = one row per patient × timepoint
    pivot = sub.pivot_table(index=['patient_id', 'time_point'],
                            columns='ai_derived', values='ratio').dropna()
    if pivot.shape[0] < 3:
        continue

    pivot.columns = ['Manual', 'AI']
    pivot['diff'] = pivot['AI'] - pivot['Manual']
    pivot['mean'] = pivot[['Manual', 'AI']].mean(axis=1)

    # --- Classic LoA ---
    mean_diff = pivot['diff'].mean()
    sd_diff = pivot['diff'].std()
    loa_lower = mean_diff - 1.96 * sd_diff
    loa_upper = mean_diff + 1.96 * sd_diff

    # --- Model-based LoA (Mixed Effects Model) ---
    df_ba = pivot.reset_index()
    df_model = pd.merge(sub,
                        df_ba[['patient_id', 'time_point', 'diff', 'mean']],
                        on=['patient_id', 'time_point'], how='inner')

    try:
        mdl = smf.mixedlm("diff ~ C(time_point)", df_model, groups="patient_id")
        res = mdl.fit(reml=True, maxiter=1000)

        model_mean_diff = res.params['Intercept']
        var_random = res.cov_re.iloc[0, 0]
        var_resid = res.scale
        total_sd = np.sqrt(var_random + var_resid)

        model_loa_lower = model_mean_diff - 1.96 * total_sd
        model_loa_upper = model_mean_diff + 1.96 * total_sd

        # Outliers check
        outliers = pivot[(pivot['diff'] > model_loa_upper) |
                         (pivot['diff'] < model_loa_lower)]
        outlier_count = len(outliers)
        total = len(pivot)
        outlier_pct = 100 * outlier_count / total

        print(f"\n{region}: {outlier_count}/{total} "
              f"({outlier_pct:.2f}%) outliers beyond model-based LoA")

        # --- Plot ---
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.scatter(df_model['mean'], df_model['diff'],
                   color='#1878d3ff', alpha=0.6)

        ax.axhline(model_mean_diff, color='black', linestyle='--',
                   label=f"Mean diff = {model_mean_diff:.4f}", linewidth=1.5)
        ax.axhline(model_loa_upper, color='black', linestyle='--',
                   label=f"Upper LoA = {model_loa_upper:.4f}", linewidth=1.0)
        ax.axhline(model_loa_lower, color='black', linestyle='--',
                   label=f"Lower LoA = {model_loa_lower:.4f}", linewidth=1.0)

        ax.set_title(f"{region} — Model-based", fontsize=14)
        ax.set_xlabel("Mean of AI and Manual", fontsize=12)
        ax.set_ylabel("Difference (AI − Manual)", fontsize=12)
        ax.set_xlim(global_xlim)
        ax.set_ylim(global_ylim)

        ax.legend(loc="upper right", fontsize=12)
        ax.text(0.95, 0.05,
                f"Outliers: {outlier_count}/{total} ({outlier_pct:.2f}%)",
                ha="right", va="bottom", transform=ax.transAxes,
                fontsize=12, bbox=dict(facecolor="white", alpha=0.7, edgecolor="none"))

        fig.tight_layout()
        fig.savefig(os.path.join(plot_dir,
                    f"{region}_model_ba.png"), dpi=300)
        plt.close()

    except Exception as e:
        print(f"{region}: Model fitting failed → {e}")
        model_mean_diff = model_loa_lower = model_loa_upper = total_sd = np.nan
        outlier_count = outlier_pct = np.nan

    # --- Save results ---
    ba_results.append({
        'Region': region,
        'Classic Mean diff': round(mean_diff, 4),
        'Classic LoA lower': round(loa_lower, 4),
        'Classic LoA upper': round(loa_upper, 4),
        'Classic SD': round(sd_diff, 4),
        'Model Mean diff': round(model_mean_diff, 4),
        'Model LoA lower': round(model_loa_lower, 4),
        'Model LoA upper': round(model_loa_upper, 4),
        'Model SD': round(total_sd, 4),
        'Outliers': outlier_count,
        'Outlier %': round(outlier_pct, 2) if pd.notnull(outlier_pct) else 'N/A'
    })

# -----------------------------
# Final Results Table
# -----------------------------
ba_df = pd.DataFrame(ba_results)
print("\n=== Bland–Altman Results Summary ===")
print(ba_df)
